## Conexión a YouTube DATA API

Para conectarnos a la API de YouTube vamos a utilizar la librería
[google-api-python-client](https://github.com/googleapis/google-api-python-client).
Con ella vamos a establecer el cliente de conexión a YouTube Data API v3
muy fácilmente. La documentación de esta librería, así como la de la API
de YouTube se puede encontrar en los siguientes links:

- [google-api-python-client](https://googleapis.github.io/google-api-python-client/docs/epy/googleapiclient.discovery-module.html)
- [youtube-data-api-v3](https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.html)
- [Data API v3](https://developers.google.com/youtube/v3/docs)

Sin más, podemos proceder a establecer la conexión (recuerda
tener el fichero con los credenciales a mano):

In [1]:
import json
import googleapiclient.discovery

# API information:
api_service_name = "youtube"
api_version = "v3"

# API key:
with open("auth/youtube_credentials.json") as file:
    credentials = json.load(file)

In [3]:
# API key:
DEVELOPER_KEY = credentials["DEVELOPER_KEY"]

# API client
youtube = googleapiclient.discovery.build(
    serviceName=api_service_name,
    version=api_version,
    developerKey=DEVELOPER_KEY
)

Ahora que ya hemos establecido la conexión, podemos comenzar
a experimentar con dicha API, por ejemplo, haciendo una búsqueda
sencilla de vídeos que contengan una palabra (en este caso vamos a
buscar "Spider-Man"):

In [12]:
# Request body:
request = youtube.search().list(
    part="id,snippet",
    type='video',
    q="Spider-Man",
    videoDuration='short',
    videoDefinition='high',
    maxResults=10
)

# Run query:
response = request.execute()

In [13]:
response

{'kind': 'youtube#searchListResponse',
 'etag': 'eAAAC4d86AvqoPhDulV-IEJdTRU',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'ES',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'FgXNnnU4h0h3sjcNo30jwVVXySk',
   'id': {'kind': 'youtube#video', 'videoId': 'JfVOs4VSpmA'},
   'snippet': {'publishedAt': '2021-11-17T01:30:05Z',
    'channelId': 'UCz97F7dMxBNOfGYu3rx8aCw',
    'title': 'SPIDER-MAN: NO WAY HOME - Official Trailer (HD)',
    'description': 'We started getting visitors… from every universe. Watch the official trailer for #SpiderManNoWayHome, exclusively in movie ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/JfVOs4VSpmA/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/JfVOs4VSpmA/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/JfVOs4VSpmA/hqdefault.jpg',
      'width': 480,
      'heig

Como podemos ver, la respuesta es tipo `JSON`,
que contiene metadata, además de la lista de resultados
que se ajustan a nuestra búsqueda. Dichos resultados están
bajo la `key` "items", y por tanto podemos consultarlos
como sigue:

In [16]:
print(f"Search results = {len(response['items'])}")
print(f"First result = {response['items'][0]}")

Search results = 10
First result = {'kind': 'youtube#searchResult', 'etag': 'FgXNnnU4h0h3sjcNo30jwVVXySk', 'id': {'kind': 'youtube#video', 'videoId': 'JfVOs4VSpmA'}, 'snippet': {'publishedAt': '2021-11-17T01:30:05Z', 'channelId': 'UCz97F7dMxBNOfGYu3rx8aCw', 'title': 'SPIDER-MAN: NO WAY HOME - Official Trailer (HD)', 'description': 'We started getting visitors… from every universe. Watch the official trailer for #SpiderManNoWayHome, exclusively in movie ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/JfVOs4VSpmA/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/JfVOs4VSpmA/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/JfVOs4VSpmA/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'Sony Pictures Entertainment', 'liveBroadcastContent': 'none', 'publishTime': '2021-11-17T01:30:05Z'}}


De la inspección del primer resultado vemos que "snippet" contiene
los datos asociados al vídeo encontrado, como por ejemplo:

In [18]:
print(response["items"][0]["id"])
response["items"][0]["snippet"]

{'kind': 'youtube#video', 'videoId': 'JfVOs4VSpmA'}


{'publishedAt': '2021-11-17T01:30:05Z',
 'channelId': 'UCz97F7dMxBNOfGYu3rx8aCw',
 'title': 'SPIDER-MAN: NO WAY HOME - Official Trailer (HD)',
 'description': 'We started getting visitors… from every universe. Watch the official trailer for #SpiderManNoWayHome, exclusively in movie ...',
 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/JfVOs4VSpmA/default.jpg',
   'width': 120,
   'height': 90},
  'medium': {'url': 'https://i.ytimg.com/vi/JfVOs4VSpmA/mqdefault.jpg',
   'width': 320,
   'height': 180},
  'high': {'url': 'https://i.ytimg.com/vi/JfVOs4VSpmA/hqdefault.jpg',
   'width': 480,
   'height': 360}},
 'channelTitle': 'Sony Pictures Entertainment',
 'liveBroadcastContent': 'none',
 'publishTime': '2021-11-17T01:30:05Z'}

Ahora que tenemos el ID de los vídeos que hemos buscado
y sus características (a partir de la info en "snippet")
podemos proceder a bajarnos la información referente a un
vídeo en particular. Tomando el ID del ejemplo anterior:


In [19]:
video_id = response["items"][0]["id"]["videoId"]

In [26]:
request = youtube.videos().list(
    id=video_id,
    part="id,statistics,snippet",
)

In [27]:
response = request.execute()

In [28]:
response

{'kind': 'youtube#videoListResponse',
 'etag': 'xbsSwl9oYGzcnW8uYPlG00ikACw',
 'items': [{'kind': 'youtube#video',
   'etag': 'lxV_MedoYuDdlwUKBxGZYxPE0Rk',
   'id': 'JfVOs4VSpmA',
   'snippet': {'publishedAt': '2021-11-17T01:30:05Z',
    'channelId': 'UCz97F7dMxBNOfGYu3rx8aCw',
    'title': 'SPIDER-MAN: NO WAY HOME - Official Trailer (HD)',
    'description': "We started getting visitors… from every universe. Watch the official trailer for #SpiderManNoWayHome, exclusively in movie theaters December 17. \nTickets on sale C̶y̶b̶e̶r̶ Spider-Monday, November 29.\n\nSubscribe to the Spider-Man Movie YouTube Channel for more exclusive content: http://bit.ly/SpiderManSubscribe\n\nVisit our site: \nhttps://www.SpiderManNoWayHome.Movie\n\nFollow Us on Social:\nhttps://www.facebook.com/SpiderManMovie\nhttps://www.instagram.com/SpiderManMovie\nhttps://www.twitter.com/SpiderManMovie\nhttps://www.tiktok.com/@SpiderManMovie \nhttps://www.youtube.com/SpiderMan\n\nFor the first time in the cinematic 

Explorando la respuesta podemos ver rápiamente
sus estadísticas, en cuanto a *likes*, etc:

In [34]:
video_data = response["items"][0]
video_data["statistics"]

{'viewCount': '71166552',
 'likeCount': '2896583',
 'favoriteCount': '0',
 'commentCount': '180999'}

Además de las estadísticas a nivel genérico, podríamos estar
interesados en conocer los comentarios que se están haciendo.
Para ello podemos hacer:

In [35]:
token = None
comments = []
for i in range(1,10):
    try:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            order="relevance",
            pageToken=token,
            maxResults=100,
        ).execute()
        token = request["nextPageToken"]
        comments += request["items"]
    except:
        print("No more comments...")

In [37]:
print(f"Hemos conseguido: {len(comments)} comentarios")

Hemos conseguido: 899 comentarios


Ejemplo de uno de los comentarios:

In [57]:
comment = comments[0]["snippet"]

print(f'Comentario:\n'
      f'autor -> {comment["topLevelComment"]["snippet"]["authorDisplayName"]}\n'
      f'autor channel-id -> {comment["topLevelComment"]["snippet"]["authorChannelId"]["value"]}\n'
      f'texto -> {comment["topLevelComment"]["snippet"]["textOriginal"]}\n'
      f'likes -> {comment["topLevelComment"]["snippet"]["likeCount"]}')

Comentario:
autor -> Holden Hardman
autor channel-id -> UCv7Qg64T82OH-EkC8t9FQbA
texto -> I haven't been this excited for a movie in such a long time. I'm so ready for this!
likes -> 41748


Se puede comprobar fácilmente que no solo tenemos el texto del
comentario, sino también métricas sobre cómo éste está siendo recibido
por el público. Además, podemos identificar el usuario que ha publicado
dicho comentario, así como su canal. Esto nos sirve para hacer un rastreo que
nos informe de la popularidad de dicho canal, etc. Para ello, vamos a usar
el ejemplo de arriba:

In [61]:
channel_id = comment["topLevelComment"]["snippet"]["authorChannelId"]["value"]
request = youtube.channels().list(
    id=channel_id,
    part="id,snippet,statistics",
)

response = request.execute()

In [65]:
response["items"][0]["statistics"]

{'viewCount': '54981640',
 'subscriberCount': '228000',
 'hiddenSubscriberCount': False,
 'videoCount': '785'}

O podemos hacer lo mismo, pero para una serie de canales:


In [66]:
YOUTUBERS = [
    "elrubiusOMG",
    "VEGETTA777",
    "Mikecrack",
    "AuronPlay",
    "TheWillyRex",
    "Makiman131",
]

channel_data = []
for user_name in YOUTUBERS:
    try:
        data = youtube.channels().list(
            part="id,snippet,statistics",
            forUsername=user_name,
        ).execute()["items"]
        channel_data += data
        print(f"user: {user_name} => subscribers: {data[0]['statistics']['subscriberCount']}")
    except:
        print(f"No data found: {user_name}")



user: elrubiusOMG => subscribers: 40400000
user: VEGETTA777 => subscribers: 32900000
No data found: Mikecrack
user: AuronPlay => subscribers: 28900000
user: TheWillyRex => subscribers: 18600000
user: Makiman131 => subscribers: 14800000
